In [ ]:
import sys
sys.path.append('sources')
from cpi_to_spin.cpitospin import create_cpi_visualization, create_spin_visualization

# Choose the test
All the test available are in folder 'CPIs'

In [ ]:
process_name = "test1" #"" #choice-task-init-reverse

import traceback
import os
import json
from cpi_to_spin.cpitospin import analyze_cpi_structure, CPIToSPINConverter

def load_cpi_file(process_name:str):
	cpi_file_path = f'CPIs/{process_name}.cpi'

	print(f"Loading CPI file: {cpi_file_path}")

	try:
		with open(cpi_file_path, 'r') as f:
			cpi_dict = json.load(f)

		print("✓ CPI file loaded successfully!")
		print(f"Root region type: {cpi_dict['type']}")
		print(f"Root region ID: {cpi_dict['id']}")

		# Pretty print the CPI structure
		print("\nCPI Structure:")
		print("=" * 50)
		print(json.dumps(cpi_dict, indent=2))
		return cpi_dict

	except FileNotFoundError:
		print(f"File not found: {cpi_file_path}")
		print("Available files in CPIs directory:")
		try:
			for f in os.listdir('CPIs'):
				if f.endswith('.cpi'):
					print(f"  - {f}")
		except:
			print("  Could not list CPIs directory")
	except Exception as e:
		print(f"❌ Error loading CPI file: {e}")
		traceback.print_exc()

	return None


def analize_cpi(cpi_dict):
	print("\nCPI Structure Analysis:")
	print("=" * 50)
	if 'cpi_dict' in locals():
		analyze_cpi_structure(cpi_dict)

	print("Converting CPI to SPIN...")
	print("=" * 50)

	try:
		converter = CPIToSPINConverter()
		spin_model = converter.convert_cpi_to_spin(cpi_dict)

		print("✓ Conversion successful!")
		print("\nSPIN Model Summary:")
		print("-" * 30)
		spin_model.print_model_summary()
		return spin_model

	except Exception as e:
		print(f"❌ Conversion failed: {e}")
		traceback.print_exc()
	return None

cpi_dict = load_cpi_file(process_name)
if cpi_dict:
	spin_model = analize_cpi(cpi_dict)

In [ ]:
cpi_viz = create_cpi_visualization(cpi_dict)
display(cpi_viz)
cpi_output = process_name + '_cpi'
cpi_viz.render(cpi_output, cleanup=True)

# FROM CPI TO SPIN

## Process Visualization

The CPI dictionary can be visualized as a directed graph to better understand its structure. In this visualization:

- **Task nodes** show duration and impact values (cost, time, quality)
- **Nature nodes** display their probability values (e.g., "p=0.7")
- **Sequence nodes** connect components with "head" and "tail" edges
- **Parallel nodes** show concurrent branches with "first" and "second" edges
- **Choice nodes** represent decision points with "true" and "false" branches

Each node type is represented as a box, with edges showing the relationships between components. This hierarchical representation helps understand the process flow and decision points in the system.

In [ ]:
spin_viz = create_spin_visualization(spin_model)
display(spin_viz)
spin_output = process_name + '_spin'
spin_viz.render(spin_output, cleanup=True)

# FROM SPIN TO PRISM

In [ ]:
prism_model = spin_model.generate_prism_model()
lines = prism_model.split('\n')

output_file =  "models/" + process_name + '.nm'
with open(output_file, 'w') as f:
	f.write(prism_model)

print("PRISM model generated successfully!")
print(f"\nFirst 20 lines of PRISM model:")
print("-" * 40)
for i, line in enumerate(lines[:20]):
	print(f"{i + 1:2d}: {line}")

if len(lines) > 20:
	print(f"... ({len(lines) - 20} more lines)")


## RUN PRISM ANALYSIS

In [ ]:
from prism import run_prism_analysis

result = run_prism_analysis(process_name, create_mdp=True)

# FROM PRISM TO MDP

Since PRISM models are based on an extended form of MDPs, we now provide the compact version of the MDP generated by PRISM, which corresponds to the equivalent SPIN model. This enhances clarity and facilitates easier comparison between the two representations.

In [ ]:
from prism_model_to_mdp.create_mdp import create_states_mdp
from prism_model_to_mdp.etl import load_prism_model
from graphviz import Source

states, transitions, rewards = load_prism_model(process_name)

compressed_dot = create_states_mdp(states, transitions, rewards)

#Source(compressed_dot).render(filename=f"models/{process_name}_cleaned", format='svg', cleanup=True)
Source(compressed_dot)